## BIOS6642  Class Project
### Kirsten Arnold

In [94]:
import sys, os
from statistics import mean, median
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randrange
from datetime import date, datetime
import csv
import string

def login():
    '''
    Allows user to input username and password. 
    Exits program after 3 failed attempts.
    '''
    user = input('Username: ')
    password = input('Password: ')
    attempts = 0  #Start at attempts = 0
    while attempts < 3:
        if user != 'BIOS6642' or password != 'python':
            attempts += 1  #increase attempts by one if either username or password are incorrect.
            print(f'Username and/or password is incorrect. {3-attempts} attempts remaining.')
            if attempts == 3: #If you have given 3 incorrect attempts, the program will close.
                sys.exit()
            user = input('Username: ') #try again with username and password
            password = input('Password: ')
        elif user == 'BIOS6642' and password == 'python': #If you have the correct username and password:
            break                                         #Break the while loop
    if attempts == 3:
        print('3 failed attempts.')
        sys.exit()                        #Program will close if you have 3 failed attempts.
    print('Login Successful.')            #If the program didn't stop, then you have succeeded at logging in!

#########################################################
    
class BankAccount:
    """ A bank account management system that will let you 
    deposit, withdraw, or transfer money to another account. """
    
    # Please delete the pass statement and implement the __init__ method.
    # This method should be used to initialize the instance variables.
    # Add proper formal parameters for the function if needed.
    
    def __init__(self, account_number, name, age, residency, balance, date_created):
        """
        Initialize the instance variables of a bank account object the following:
        -Unique account number between 100 and 99999 (including 100 and 99999)
        -Account owner name (string)
        -Account owner age (positive integer)
        -Account owner state of residency (string)
        -Current balance (positive float)
        -Account creation date (string = 'month/day/year')
        """
        if account_number in range(100,100000):
            self.__account_number = account_number #account number must be between numbers 100 and 99999
        self.__name = name.title()                 #converts name to title case if not already
        self.__age = age
        self.__residency = residency.title()       #converts residency to title case if not already
        self.__balance = round(balance,2)
        self.__date_created = date_created
        
        #Now add this account and all information to a dictionary
        try: #If account number is not already in use, add it to the class dictionary
            if self.__account_number not in BankAccount.__dct:
                BankAccount.__dct[self.__account_number] = [self.__name, self.__age, self.__residency, 
                                                self.__balance, self.__date_created]
        except AttributeError:
            #If BankAccount._dct doesn't exist, create empty dict then add info to dict
            BankAccount.__dct = {}  
            BankAccount.__dct[self.__account_number] = [self.__name, self.__age, self.__residency, 
                                                self.__balance, self.__date_created]
    
    @property
    def account_number(self):
        '''Returns the account number.'''
        return self.__account_number
    
    @property
    def name(self):
        '''Returns the name of account owner.'''
        return self.__name
     
    @name.setter
    def name(self,new_name):
        '''Sets a new name of account owner if their name changes.'''
        self.__name = new_name.title()
        BankAccount.__dct[self.__account_number][0] = self.__name
    
    @property
    def age(self):
        '''Returns the age of account owner.'''
        return self.__age
    
    @age.setter
    def age(self,new_age):
        '''Set a new age/updates age of account owner.'''
        self.__age = new_age #update the attribute of the class object
        BankAccount.__dct[self.__account_number][1] = self.__age #update the age in the class dictionary
        
    @property
    def residency(self):
        '''Returns state of account owner's residency.'''
        return self.__residency
    
    @residency.setter
    def residency(self, new_residency):
        '''Update residency of account owner.'''
        self.__residency = new_residency            #update the attribute of the class object
        BankAccount.__dct[self.__account_number][2] = self.__residency #update the residency in the class dct
    
    @property
    def balance(self):
        '''Returns current balance of account.'''
        return round(self.__balance,2)

    @property
    def date_created(self):
        '''Returns date the account was created.'''
        return self.__date_created

            
#########################################################
    
    @classmethod
    def all_accounts(cls):
        '''Returns a dictionary of all accounts.'''  
        #Makes it so you can use items in class dct for other functions, outside of class methods
        return BankAccount.__dct
            
#########################################################
    
    @classmethod
    def deposit(cls):
        """ Add a certain amount of money to an account. """
        
        print()
        try:
            account_number = int(input('Please enter account number for deposit: '))
            deposit = round(float(input('Please enter amount to deposit: ')),2)
            balance = BankAccount.__dct[account_number][3] 
        except ValueError: #Happens if you don't input an integer for the account number or float for deposit
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        except KeyError: #If account isn't in dictionary and raises an error
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        
        #If an exception is raised, it will not run the code below this
        new_balance = round(balance + deposit,2)
        globals()[f'account_{account_number}'].__balance = new_balance #update attribute for class object
        BankAccount.__dct[account_number][3] = new_balance #update new balance in dict
        
        #Update new balance in account_data.csv file
        with open('account_data.csv','r') as infile:
            records = []
            lines = infile.readlines()
            for line in lines:
                values = line.strip().split(',') #each value in each line of the csv file is split by commas
                if values[0] != f'{account_number}': #if not the account in question, no changes are made
                    records.append(values)           #and add the account info to the list of records
                else:
                    values[4] = f'{new_balance}' #if it is the account in question, update the balance
                    records.append(values)       #then add the account info to the list of records
        with open('account_data.csv','w') as outfile:
            for record in records:
                csv.writer(outfile).writerow(record) #rewrite the account data file one row/record at a time
        
        #Record transaction info in account_transactions.csv file
        transaction = [str(account_number), 'Deposit of $'+str(deposit)+'.'] #Transaction info (account#, action)
        with open('account_transactions.csv','a') as f: #add transaction info to the end of the existing file
            csv.writer(f).writerow(transaction)
        print('\nTransaction successful. Returning to main menu.')
        operations() #return to main menu

    @classmethod
    def withdraw(cls):
        """ Remove a certain amount of money from an account. """
        
        print()
        try:
            account_number = int(input('Please enter account number for withdrawal: '))
            withdrawal = round(float(input('Please enter amount to withdraw: ')),2)
            balance = BankAccount.__dct[account_number][3]
        except ValueError: #If someone doesn't put an int as account# or float as withdrawal input
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        except KeyError: #If account isn't in dictionary
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        
        #If an exception is raised, it will not run the code below this
        if balance - withdrawal < 0: #if account has insufficient funds for withdrawal
            print('\nInsufficient funds. Transaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        else: #if they do have sufficient funds
            new_balance = round(balance - withdrawal,2)
            globals()[f'account_{account_number}'].__balance = new_balance #update balance attribute
            BankAccount.__dct[account_number][3] = new_balance #update balance in class dct
            
            #update account data file
            with open('account_data.csv','r') as infile:
                records = []
                lines = infile.readlines()
                for line in lines:
                    values = line.strip().split(',')
                    if values[0] != f'{account_number}': #if not the account in question,
                        records.append(values)           #no changes are made to the record 
                    else:
                        values[4] = f'{new_balance}'     #if it IS the account in question, update balance
                        records.append(values)           #and add to the record
            with open('account_data.csv','w') as outfile:
                for record in records:
                    csv.writer(outfile).writerow(record) #rewrite the account data file one row/record at a time
            
            #Record transaction info in account_transactions.csv file
            #transaction info = [account number, action]
            transaction = [str(account_number), 'Withdrawal of $'+str(withdrawal)+'.']
            with open('account_transactions.csv','a') as f:
                csv.writer(f).writerow(transaction) #add transaction info to the end of the existing file
            print('\nTransaction successful. Returning to main menu.')
            operations() #return to main menu

    @classmethod
    def transfer(cls):
        """ 
        Transfer a certain amount of money from an account to another account. 
        """
        print()
        try:
            sender = int(input('Please enter account number of sender: '))
            receiver = int(input('Please enter account number of recipient: '))
            amount = round(float(input('Please enter amount to be transferred: ')),2)
            print(f'Attempting transfer of ${amount} from account# {sender} to account# {receiver} :')
            sender_balance = BankAccount.__dct[sender][3]
            receiver_balance = BankAccount.__dct[receiver][3]
        except ValueError: #If wrong input is entered
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        except KeyError: #If account isn't in dictionary and raises an error
            print('\nTransaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        
        #If any of the exceptions above are triggered, everything below this line is not run
        if sender_balance - amount < 0: #If there isn't enough money to transfer
            print('\nInsufficient funds. Transaction unsuccessful. Returning to main menu.')
            operations() #return to main menu
        else:
            sender_new_balance = round(sender_balance - amount,2)
            receiver_new_balance = round(receiver_balance + amount,2)
            globals()[f'account_{sender}'].__balance = sender_new_balance #update sender's balance attribute
            globals()[f'account_{receiver}'].__balance = receiver_new_balance #update recipient's balance attribute
            BankAccount.__dct[sender][3] = sender_new_balance        #Update class dictionary entry for sender
            BankAccount.__dct[receiver][3] = receiver_new_balance    #Update class dictionary entry for receiver
            
            #Update the account_data.csv file
            with open('account_data.csv','r') as infile:
                records = []
                lines = infile.readlines()
                for line in lines:
                    values = line.strip().split(',')
                    if values[0] == f'{sender}':            #if sender account, 
                        values[4] = f'{sender_new_balance}' #update their new balance
                        records.append(values)              #then add updated account info to records list
                    elif values[0] == f'{receiver}':          #If receiver account, 
                        values[4] = f'{receiver_new_balance}' #update their new balance and
                        records.append(values)                #add updated account info to records list
                    else:
                        records.append(values)     #If account is neither sender nor receiver, no updates needed
            with open('account_data.csv','w') as outfile:
                for record in records:
                    csv.writer(outfile).writerow(record)
            
            #Record transaction info for both sender and receiver in transaction info file
            #Transaction info = [account#, action]
            transaction_sender = [str(sender), 'Transfer of $'+str(amount)+' to account# '+str(receiver)+'.']
            transaction_receiver = [str(receiver), 'Received transfer of $'+str(amount)+' from account# '+str(sender)+'.']
            with open('account_transactions.csv','a') as f:
                csv.writer(f).writerow(transaction_sender)    #Add sender transaction info to end of file
                csv.writer(f).writerow(transaction_receiver)  #Add receiver transaction info to end of file
            print('\nTransfer successful. Returning to main menu.')
            operations() #return to main menu


#########################################################            

    @classmethod
    def print_sorted_accounts(cls, sortby=1):
        '''
        Prints out tabular table of all accounts.
        Can sort by 1)account number [default], 2)name, 3)current balance, or 4)account creation date.
        '''
        
        print()
        dct = BankAccount.__dct 
        if sortby == 1: #if sorting by account number
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',')  #first print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in sorted(dct): #sorting by account number is just sorting by dict keys
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*dct[k])) #tabular format
        elif sortby == 2: #if sorting by name
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',')  #first print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in sorted(dct.keys(), key = lambda x:dct[x]): #sorts by name (first element in dct values)
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*dct[k])) #tabular format
        elif sortby == 3: #if sorting by balance
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',')  #first print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in sorted(dct.keys(), key = lambda x:dct[x][3]): #sorts by balance (4th element in dct values)
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*dct[k])) #tabular format
        elif sortby == 4: #if sorting by date
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',')  #first print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            #convert dates (5th element in dct values) to datetime format then sorts them by date
            for k in sorted(dct.keys(), key = lambda x:datetime.strptime(dct[x][4], '%m/%d/%y')):
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*dct[k])) #tabular format
        else: #if any selection other than 1, 2, 3, or 4
            print('Invalid selection.')
        operations() #return to main menu

#########################################################
        
    @classmethod
    def stratify(cls, selection):
        '''
        Stratifies account statistics (total, mean, and median) based on selection: 
        1) all accounts, 2)by age group, and 3)by state/residency. 
        '''
        
        print()
        dct = BankAccount.__dct
        balances = []
        for item in dct:
            balances.append(dct[item][3]) #Make a list with all balances from bank accounts dictionary
        if selection == 1: #All accounts statistics, rounded by 2 decimal points
            print(f'All accounts \nTotal Balance: ${round(sum(balances),2)},' 
                  f'\nAverage Balance: ${round(mean(balances),2)}',
                 f'\nMedian Balance: ${round(median(balances),2)}')
        
        elif selection == 2: #if you stratify by age group
            balances = np.array(balances) #convert list of balances to array for mask/filtering purposes
            ages = []
            for item in dct:
                ages.append(dct[item][1]) #make a list of all ages associated with each account
            ages = np.array(ages)         #convert list of ages to array for mask/filtering purposes
            
            #Make a list of all calculated totals, using each age group as a mask for the balances array
            group1 = [np.sum(balances[ages < 35]),
                      round(np.mean(balances[ages < 35]),2), 
                      round(np.median(balances[ages < 35]),2)]
            group2 = [np.sum(balances[np.logical_and(ages >= 35, ages <=44)]),
                    round(np.mean(balances[np.logical_and(ages >= 35, ages <=44)]),2),
                    round(np.median(balances[np.logical_and(ages >= 35, ages <=44)]),2)]
            group3 = [np.sum(balances[np.logical_and(ages >= 45, ages <=54)]),
                    round(np.mean(balances[np.logical_and(ages >= 45, ages <=54)]),2),
                    round(np.median(balances[np.logical_and(ages >= 45, ages <=54)]),2)]
            group4 = [np.sum(balances[np.logical_and(ages >= 55, ages <=64)]),
                    round(np.mean(balances[np.logical_and(ages >= 55, ages <=64)]),2),
                    round(np.median(balances[np.logical_and(ages >= 55, ages <=64)]),2)]
            group5 = [np.sum(balances[np.logical_and(ages >= 65, ages <=74)]),
                    round(np.mean(balances[np.logical_and(ages >= 65, ages <=74)]),2),
                    round(np.median(balances[np.logical_and(ages >= 65, ages <=74)]),2)]
            group6 = [np.sum(balances[ages >= 75]),round(np.mean(balances[ages >= 75]),2),
                    round(np.median(balances[ages >= 75]),2)]
            
            #Make a dict for each age group with total, mean, and median as the values for the age groups
            age_dct = {'<35 Years Old': group1, '35-44 Years Old': group2, '45-54 Years Old': group3,
                    '55-64 Years Old': group4, '65-74 Years Old': group5,'75 Years or Older': group6}
            
            #Make empty dataframe with labeled columns then add rows for each entry in age_dct
            data = pd.DataFrame(columns = ['Total','Average','Median'])
            for k in age_dct.keys():
                data.loc[k] = age_dct[k]
            print(data)
            
            #Plot data by age group in a line/curve
            data['Average'].plot(label = 'Average') #plotting average then
            data['Median'].plot(label = 'Median')   #plotting median on same plot
            plt.legend(loc='lower right')           #adding legend to lower right
            plt.ticklabel_format(axis='y', style='plain') #Changing y-axis from scientific notation to standard
            plt.ylabel('Balance (USD)')
            plt.xlabel('Age Group')
            plt.title('Average/Median Balance by Age Group')
            plt.xticks(rotation=90) #Rotate x-axis labels by 90 degrees for easy readability
            plt.show()
                
        elif selection == 3: #if you stratify by state of residence
            balances = np.array(balances) #convert list to array for masking/filtering purposes
            res = []
            data = pd.DataFrame(columns = ['Total','Average','Median']) #Empty dataframe to add rows later
            for item in dct:
                res.append(dct[item][2]) #for each account, add their residency to the list
            res = np.array(res) #convert to array for filtering purposes
            res_labels = np.unique(res) #Make an array for each unique state/residence
            for state in res_labels: #For each unique state, use res array to filter the balances
                total = np.sum(balances[res == state]) #then calculate total, mean, and median balances
                avg = round(np.mean(balances[res == state]),2)  #for those states
                med = round(np.median(balances[res == state]),2)
                data.loc[state] = [total, avg, med] #then add the state as the index plus values to dataframe 
            print(data)
            
            #plot Balance by state of residency
            df = pd.DataFrame(data, columns = ['Average','Median']) #use average and median from dataframe
            df.plot.bar() #plot the averages and medians of each state as side-by-side bars
            plt.ylabel('Balance (USD)')
            plt.xlabel('State of Residency')
            plt.title('Balance by State of Residency')
            plt.xticks(rotation=90) #rotate x-axis labels by 90 degrees for readability
            plt.ticklabel_format(axis='y', style='plain') #Show balance in standard format instead of scientific notation
            plt.show()
        else: #if 1, 2, or 3 were not provided as the selection
            print('Selection not recognized. Returning to main menu.')
        operations() #return to main menu

#########################################################
    
    @classmethod
    def subset(cls, subset_by):
        '''
        Shows all accounts within 
        1)a specified age range, 
        2)a specified state of residence, or 
        3)a specified date range for account creation.
        '''
        
        print()
        dct = BankAccount.__dct
        new_dct = {}
        if subset_by == 1: #subset by age within an age range
            try:
                min_age = int(input('Subset accounts by age -- Enter minimum age: '))
                max_age = int(input('Subset accounts by age -- Enter maximum age: '))
                if min_age > max_age: #if for some reason someone can't read and switches the min/max age,
                    min_age, max_age = max_age, min_age #switch them back
            except ValueError: #if someone decides not to put age as an integer and raises this error
                print('\nInput not accepted. Age must be given as an integer. Please try again.')
                BankAccount.subset(1) #run the subset program again
            print()
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',') #print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in dct:                                 #for each account
                if dct[k][1] in range(min_age,max_age+1): #if it's between min and max age in range,
                    new_dct[k] = dct[k]                   #copy that dct entry to a new dct
            for k in sorted(new_dct): #print each new_dct entry account info in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*new_dct[k]))
        
        elif subset_by == 2: #subset by residency
            state = (input('Subset by residency -- Enter state of residency: ')).title()
            print()
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',') #print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in dct:               #for each account
                if state == dct[k][2]:  #if that account has the specified state of residency
                    new_dct[k] = dct[k] #copy that dct entry to a new dct
            for k in sorted(new_dct): #then print each new_dct entry account info in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*new_dct[k]))
        
        elif subset_by == 3: #subset by creation date in range
            try:
                min_date = datetime.strptime(str(input(
                    'Subset accounts by creation date -- Enter earliest date (mm/dd/yy): ')),'%m/%d/%y')
                max_date = datetime.strptime(str(input(
                    'Subset accounts by creation date -- Enter latest date (mm/dd/yy): ')),'%m/%d/%y')
            except ValueError: #if date range inputs aren't in the right format/value
                print('\nDates must be entered as mm/dd/yy. Please try again.')
                BankAccount.subset(3) #rerun the subset program for subset_by = 3
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',') #print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            for k in dct: #for each account, if it is within the specified date range
                if datetime.strptime(dct[k][4],'%m/%d/%y') >= min_date and datetime.strptime(dct[k][4],'%m/%d/%y') <= max_date:
                    new_dct[k] = dct[k] #copy that dct entry to a new dct
            for k in sorted(new_dct):   #then print each new_dct entry account info in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} ${:<11} {:<10} >'.format(k,*new_dct[k]))
        
        #if subset_by selection input from user is anything other than 1, 2, or 3
        else: print('Selection not recognized. Returning to main menu.')
        operations() #return to main menu
        
#########################################################    
                
    @classmethod
    def account_lookup(cls,account_num):
        '''
        Returns all info related to account given the account number, including:
        name, age, residency, balance, and date account was created.
        '''
        
        print()
        if account_num in BankAccount.__dct:    #if the account exists,
            with open('account_data.csv') as f:
                header = f.readline()
                colnames = header.strip().split(',') #print header column names in tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames))
            #then print all account info from dictionary in a tabular format
            print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(account_num,*BankAccount.__dct[account_num]))
            operations() #then return to main menu
        else: 
            print('Account does not exist. Please try again.')
            BankAccount.query() #if account doesn't exist, go back to query menu
        
            

    @classmethod
    def name_lookup(cls, name):
        '''
        Returns all account information for owners with the given name.
        '''
        
        print()
        entry = False #we have not found an entry with the provided name unless specified otherwise
        for k in BankAccount.__dct: #for each account
            if BankAccount.__dct[k][0] == name.title(): #if that dct entry has the specified name
                if entry == False: #If it's the first time name was found, print column headings
                    with open('account_data.csv') as f:
                        header = f.readline()
                        colnames = header.strip().split(',')
                        print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(*colnames)) #tabular format
                entry = True #found a dictionary entry with the name
                #then print account info in a tabular format
                print('< {:<15} {:<10} {:<4} {:<12} {:<12} {:<10} >'.format(k,*BankAccount.__dct[k]))
        if entry == False: #if after you go through every account in dct and entry is still false
            print('Account does not exist. Please try again.')
            BankAccount.query() #go back to query menu
        operations()
                
    @classmethod
    def query(cls):
        '''
        Asks user for input to look up account and offers the option to return to the main menu or quit.
        This class method will determine if they look up the program by account number or account name.
        '''
        account = input('Please enter account number or name of account owner'+
                        '(type "m" to return to main menu, or "q" to quit): ')
        if account == "m":
            operations() #return to main menu
        elif account == "q":
            sys.exit()   #immediately quit the program
        elif account.isdigit(): #if they provide an account number (only contains digits)
            account = int(account) #convert it to an integer
            BankAccount.account_lookup(account) #and look up account by account number
        elif account.isalpha(): #if only contains letters of the alphabet
            account = account.title() #convert to title case
            BankAccount.name_lookup(account) #lookup account by name
        else: #if not an integer or alphabetic, must have other special characters
            print('Input not recognized as account number or name. Try again.')
            BankAccount.query() #return to query menu
            
        
        

#########################################################                

    @classmethod
    def open_account(cls):
        '''
        Opens a new account. 
        Automatically provides a random account number between 100 - 99,999 and 
        records the date of creation as the current date.
        Asks for user input for name, age, residency, and balance.
        Also updates account_info.csv file.
        '''
        
        account_number = randrange(100, 100000) #automatically generates new account num between 100 & 99,999
        while account_number in BankAccount.__dct: #while account_number is already in dictionary
            account_number = randrange(100, 100000) #generate a new random account number 
        name = input('Please enter name of new account owner: ').title() #Asks user for name of new account owner
        try:
            age = int(input('Please enter age of account owner: ')) #Asks user for age of new account owner
        except ValueError: #if can't be converted to integer
            print('Age must be an integer. Account creation failed. Returning to main menu.')
            operations() #return to main menu
        residency = input('Please enter state of residency: ').title() #Asks user for residency of new account owner
        try: balance = round(float(input('Please enter starting account balance: ')),2) #Asks user for balance of account
        except ValueError: #if it can't be converted to a float
            print('Starting balance must be a number. Returning to main menu.')
            operations() #return to main menu
        date_created = date.today().strftime('%m/%d/%y') #Automatically makes creation date as current date
        globals()[f'account_{account_number}'] = BankAccount(account_number,name,age,residency,balance,date_created)
        #creates BankAccount object, automatically updates class dct
        
        #Add new account information to end of account_data.csv file on new line
        with open('account_data.csv','a') as f: #append to add to end of existing file
            account_record = [str(account_number),name,str(age),residency,str(balance),str(date_created)]
            csv.writer(f).writerow(account_record) #Add account record to new row in csv file
        print('Account creation successful.',
              f'Account#: {account_number}, name: {name}, age: {age}, residency: {residency}, starting balance: ${balance}, date created: {date_created}.',
              'Returning to main menu.', sep = '\n')
        operations() #return to main menu
    
#########################################################    

    @classmethod
    def del_account(cls):
        '''
        Deletes account from BankAccount class, the class dictionary, and the account information file.
        '''
        try:
            account_num = int(input('Please enter account number to be deleted: '))
        except ValueError: #if it can't be converted to an integer
            print('Input not recognized. Returning to main menu.')
            operations() #return to main menu
        while account_num not in BankAccount.__dct: #while provided account number is not in the dct
            account_num = input('Account not in database.'+                         
                                'Please enter another account number or "q" to quit '+ #ask again
                                'or "m" for main menu: ') #they can also return to main menu or quit
            if account_num == "q":
                sys.exit() #immediately quit the program if they choose to quit
            if account_num == "m":
                operations() #immediately return to main menu if they choose to do so
        
        #once the provided account is recognized as being in the database
        del BankAccount.__dct[account_num] #delete the dct entry
        del globals()[f'account_{account_num}'] #delete the class object
        #delete the account from the account_data.csv file
        with open('account_data.csv','r') as infile:
            records = []
            lines = infile.readlines()
            for line in lines:                         #for each account in csv file
                values = line.strip().split(',')       #split values by commas
                if values[0] != f'{account_num}':      #if the account isn't the account in question
                    records.append(values)             #add the account info to the list of records
        with open('account_data.csv','w') as outfile:
            for record in records:                     #rewrite the account_data.csv file
                csv.writer(outfile).writerow(record)   #write one row/account at a time
        print('\nAccount deletion successful. Returning to main menu.')
        operations() #return to main menu

#########################################################    

    @classmethod
    def del_all_accounts(cls):                      
        '''
        Deletes all accounts. This method is used in the process of exiting the program
        so that class objects and class dictionary cannot be accessed outside of the program.
        '''
        if hasattr(BankAccount, '__dct'):           #If bank account dct hasn't been created yet
            for account in BankAccount.__dct:
                del globals()[f'account_{account}'] #delete each BankAccount object associated with dictionary
            del BankAccount.__dct                   #then delete the dct
        
#########################################################    

    @classmethod
    def recent_transactions(cls,account_number):
        '''
        Displays five most recent transactions associated with the given account number.
        '''
        with open('account_transactions.csv','r') as f:
            records = []
            lines = f.readlines()
            for line in lines:                       #for each transaction entry
                values = line.strip().split(',')     #split into account#, transaction
                if values[0] == f'{account_number}': #if transaction entry is for account number in question
                    records.append(values)           #add the record to the records list
        if len(records) > 5:                         #if length of records for that account are > 5
            for record in records[-5:]:           #only use last 5 records in list. For each of those records
                print('< {:<15} {:<50} >'.format(f'Account# {record[0]}',record[1])) #print in tabular format
        else: 
            for record in records: #if length is not >5, print each record in tabular format
                print('< {:<15} {:<50} >'.format(f'Account# {record[0]}',record[1]))
                

#########################################################    


def database_initialization(database_filename):
    '''
    Initializes the database using the current csv file
    '''
    BankAccount.__dct = {} #reset Bank Account dictionary
    with open(database_filename) as f:
        lines = f.readlines()
        for line in lines[1:]:
            account_num, name, age, residency, balance, date = line.strip().split(',')
            account_num = int(account_num)
            age = int(age)
            balance = float(balance)
            month, day, year = date.strip().split('/')
            month, day = month.rjust(2,'0'), day.rjust(2,'0') #adjusting format so date can be mm/dd/yy
            date = str('/'.join((month,day,year)))
            # Adds global variables for all accounts with the name "account_{number}"
            globals()[f'account_{account_num}'] = BankAccount(account_num, 
                                                              name, age, residency, balance, date)
            #This also adds all accounts from the initialization file into the BankAccount dictionary
            #When you create the class objects
    
    #For some reason, if I don't rewrite the csv file first, when I add a new account, 
    #it will not append in a new row,
    #it appends in the same row as the last row the first time this is attempted.
    #So I will rewrite the file here.
    with open('account_data.csv','r') as infile:
        records = []
        lines = infile.readlines()
        for line in lines:                         #for each account in csv file
            values = line.strip().split(',')       #split values by commas
            records.append(values)                 #add the account info to the list of records
    with open('account_data.csv','w') as outfile:
        for record in records:                     #rewrite the account_data.csv file
            csv.writer(outfile).writerow(record)   #write one row/account at a time
            
######################################################### 
    
def transactions(selection):
    '''
    Function that decides which class method to use 
    after the user provides their selection from the transactions menu.
    '''
    try:
        with open('account_transactions.csv','x') as f: #if file does not exist, create it
            header = ['Account#','Transaction Performed'] #and provide the headers
            csv.writer(f).writerow(header) #write the header as first row of csv file
    except FileExistsError: #if the file already exists, do nothing
        pass
    finally:
        if selection == 1: #if they want to make a deposit, use this method
            BankAccount.deposit()
        elif selection == 2: #if they want to make a withdrawal, use this method
            BankAccount.withdraw()
        elif selection == 3: #if they want to initiate a transfer of funds, use this method
            BankAccount.transfer()
        elif selection == 4: #if they want to look up an account balance
            try:
                number = int(input('Please enter account number: ')) #ask for the account number
            except ValueError: #if provided account number cannot be converted to an integer
                print('Invalid account number. Returning to main menu.')
                operations() #return to main menu
            #otherwise print the information requested:
            try:
                print(f'\nBalance of account# {number} is', globals()[f'account_{number}'].balance)
            except KeyError: #If account class object does not exist
                print('Account does not exist.')
                operations() #return to main menu
        elif selection == 5: #if they want to change any account info, provide another menu
        
            print('Type "1" to change name',
                  'Type "2" to change age',
                  'Type "3" to change state of residency', sep = '\n')
            try:
                revise = int(input('Selection: ')) #user makes selection for changing name/age/residency
                if revise == 1: #if they want to change the name
                    number = int(input('Please enter account number: ')) #ask for account number
                    old_name = globals()[f'account_{number}'].name #stores old name
                    new_name = input('Please enter new name: ') #asks for new name
                    globals()[f'account_{number}'].name = new_name 
                    #this automatically updates class object, class dct
                    
                    #now update the account data file
                    with open('account_data.csv','r') as infile:
                        records = []
                        lines = infile.readlines()
                        for line in lines:
                            values = line.strip().split(',') #each value in each line of the csv file is split by commas
                            if values[0] != str(number): #if not the account in question, no changes are made 
                                records.append(values) #and add the account info to the list of records
                            else:
                                values[1] = str(new_name)    #if it is the account in question, update the name
                                records.append(values)         #then add the account info to the list of records
                    with open('account_data.csv','w') as outfile:
                        for record in records:                    #rewrite the account data file one row/record at a time
                            csv.writer(outfile).writerow(record)
                    
                    #Record transaction info in account_transactions.csv file
                    #transaction info = [account number, action]
                    transaction = [str(number), 'Name changed from '+old_name+' to '+new_name+'.']
                    with open('account_transactions.csv','a') as f:
                        csv.writer(f).writerow(transaction) #add transaction info to the end of the existing file
                    print('\nTransaction successful. Returning to main menu.')
                            
                elif revise == 2: #if they want to change the age
                    number = int(input('Please enter account number: ')) #user provides account number
                    old_age = globals()[f'account_{number}'].age #stores old age
                    new_age = input('Please enter new age: ')    #asks for new age
                    globals()[f'account_{number}'].age = new_age 
                    #this automatically updates class object, class dct
                     
                    #now update the account data file
                    with open('account_data.csv','r') as infile:
                        records = []
                        lines = infile.readlines()
                        for line in lines:
                            values = line.strip().split(',') #each value in each line of the csv file is split by commas
                            if values[0] != str(number): #if not the account in question, no changes are made 
                                records.append(values) #and add the account info to the list of records
                            else:
                                values[2] = str(new_age)    #if it is the account in question, update the name
                                records.append(values)         #then add the account info to the list of records
                    with open('account_data.csv','w') as outfile:
                        for record in records:                    #rewrite the account data file one row/record at a time
                            csv.writer(outfile).writerow(record)
                            
                    #Record the transaction/name change
                    #transaction = [account#, action]
                    transaction = [str(number), 'Age changed from '+str(old_age)+' to '+str(new_age)+'.']
                    with open('account_transactions.csv','a') as f:
                        csv.writer(f).writerow(transaction) #add transaction information to end of csv file
                    print('\nTransaction successful. Returning to main menu.')
                
                elif revise == 3: #if they want to change their state of residence
                    number = int(input('Please enter account number: ')) #user provides account number
                    old_residency = globals()[f'account_{number}'].residency #stores old residency
                    new_residency = input('Please enter new state of residence: ') #asks for new residency
                    globals()[f'account_{number}'].residency = new_residency
                    #this automatically updates class object, class dct
                    
                    #now update the account data file
                    with open('account_data.csv','r') as infile:
                        records = []
                        lines = infile.readlines()
                        for line in lines:
                            values = line.strip().split(',') #each value in each line of the csv file is split by commas
                            if values[0] != str(number): #if not the account in question, no changes are made 
                                records.append(values) #and add the account info to the list of records
                            else:
                                values[3] = str(new_residency)    #if it is the account in question, update the name
                                records.append(values)         #then add the account info to the list of records
                    with open('account_data.csv','w') as outfile:
                        for record in records:                    #rewrite the account data file one row/record at a time
                            csv.writer(outfile).writerow(record)
                    
                    #Record the transaction/name change
                    #transaction = [account#, action]
                    transaction = [str(number), 'Residence change from '+old_residency+' to '+new_residency+'.']
                    with open('account_transactions.csv','a') as f:
                        csv.writer(f).writerow(transaction) #add transaction information to end of csv file
                    print('\nTransaction successful. Returning to main menu.')
                
                else:   #if input is an integer other than 1, 2, or 3
                    print('Invalid selection. Returning to main menu.') 
            
            except ValueError: #if input can't be converted to integer
                print('Invalid. Returning to main menu.')
            except KeyError: #if account isn't in the database
                print('Account not in database. Returning to main menu.')
            finally: operations() #return to main menu
        
        elif selection == 6: #if you want to look at recent transactions
            try:
                account_number = int(input('Please enter account number: ')) #ask for account in question
                BankAccount.recent_transactions(account_number) #then run this method
            except ValueError: #if provided account number can't be converted to an integer
                print('System does not recognize selection. Returning to main menu.')
                operations() #return to main menu
        else:
            print('System does not recognize selection. Returning to main menu.')
        operations() #return to main menu
            
            
            
######################################################### 

def operations():
    '''
    Provides the user with the main menu for bank operations.
    
    Operations include: 
        displaying all accounts,
        displaying account statistics,
        displaying accounts in a subset of a category,
        displaying the account information for a single account,
        opening a new account,
        deleting an existing account,
        or conducting various transactions.
    '''
    
    print('\nMain Menu',
        'Type "1" to display all accounts',
        'Type "2" for account statistics',
        'Type "3" for displaying accounts in subset of a category',
        'Type "4" for account query',
        'Type "5" to open a new account',
        'Type "6" to delete an account',
        'Type "7" to conduct a transaction (deposit, withdrawal, transfer, display balance,', 
        'change name/age/residency, display recent account activity)',
        'Type "q" to quit.', sep = '\n')
    choice = input('Selection: ') #ask user which bank operation they would like to choose
    if choice == 'q':
        sys.exit() #if they ever choose to quit while in the main menu, the program will automatically close
    try:
        choice = int(choice)
    except ValueError: #if choice is not an integer
        print('Selection not recognized. Please try Again.')
        operations() #bring up main menu again
    if choice == 1: #if they choose to display all accounts sorted by selected category
        #provide user with another menu to select which category to sort by
        print('\nType "1" to sort by account number',
          'Type "2" to sort by name',
          'Type "3" to sort by current balance',
          'Type "4" to sort by account creation date',
          'Type "q" to quit.', sep = '\n')
        sortby = input('Selection: ')
        if sortby == 'q':
            sys.exit() #if they choose to quit, program will immediately close
        try:
            sortby = int(sortby)
        except ValueError: #if sortby menu choice cannot be converted to an integer
            print('\nSelection not recognized. Returning to main menu.')
            operations() #return to main menu
        BankAccount.print_sorted_accounts(sortby) #otherwise execute this method
    
    elif choice == 2: #Show account statistics (total/avg/median) for accounts by category
        #if they choose to display account statistics by category
        #provide user with another menu to select which category to stratify by
        print('\nType "1" for total, average, and median balance of all accounts',
          'Type "2" for total, average, and median balance of accounts by age group',
          'Type "3" for total, average, and median balance of accounts by state/residency',
          'Type "q" to quit.', sep = '\n')
        selection = input('Selection: ')
        if selection == 'q':
            sys.exit() #immediately exits the program by raising SystemExit error
        else:
            try:
                selection = int(selection)
            except ValueError: #if selection cannot be converted to integer
                print('\nSelection not recognized. Returning to main menu.')
                operations() #return to main menu
            BankAccount.stratify(selection) #otherwise execute this method
    
    elif choice == 3: #Show accounts subset by category
        #if they choose to display account subset by category
        #provide user with another menu to select which category to subset by
        print('\nType "1" to subset accounts by age',
             'Type "2" to subset accounts by state/residency',
             'Type "3" to subset accounts by account creation date',
             'Type "q" to quit.', sep = '\n')
        subsetby = input('Selection: ')
        if subsetby == 'q':
            sys.exit() #immediately exits the program by raising SystemExit error
        else:
            try: subsetby = int(subsetby)
            except ValueError: #if selection cannot be converted to integer
                print('\nSelection not recognized. Returning to main menu.')
                operations() #return to main menu
            BankAccount.subset(subsetby) #otherwise execute this method
    
    elif choice == 4: #If they want to look up account information by either account number or name
        BankAccount.query() #execute this method
            
    elif choice == 5: #If they want to open new account
        BankAccount.open_account() #execute this method
    
    elif choice == 6: #If they want to delete an account
        BankAccount.del_account() #execute this method
    
    elif choice == 7: #If they want to conduct a transaction, provide user with this transaction menu
        print('\nType "1" to make a deposit',
             'Type "2" to make a withdrawal',
             'Type "3" to transfer money from one account to another',
             'Type "4" to display current balance for an account',
             'Type "5" to change any account information including name, age, or state of residency',
             'Type "6" to display the five most recent transactions for an account',
             'Type "q" to quit.', sep = '\n')
        selection = input('Selection: ')
        if selection == 'q':
            sys.exit() #immediately exits the program by raising SystemExit error
        else:
            try: selection = int(selection)
            except ValueError: #if selection cannot be converted to an integer
                print('\nSelection not recognized. Returning to main menu.')
                operations() #return to main menu
            transactions(selection) #otherwise perform this function
    else: print('\nSelection not recognized. Returning to main menu.') #if selection is not "q" or 1-7
    operations() #return to main menu
        

#########################################################

def system_start():
    '''
    Starts up bank management system.
    3 login attempts are allowed.
    Runs the system up until the user decides to quit.
    Upon program exit, the program deletes all account information stored in objects
    so that the only way to access anyone's account information is 
    from the account_data.csv file itself.
    '''
    
    #diamond logo for login screen
    for row in range(5):
        print(' '*(5-row),'*'*(row*2+1),sep='')
    for row in range(6):
        print(' '*row,'*'*((5-row)*2+1),sep='')
    
    print('\nBank Management System')
    print()
    try:
        login()                                     #attempt to login
        database_initialization('account_data.csv') #initialize database using account information file
        operations()                                #bring up the main menu
    except SystemExit:                          #Whenever a SystemExit error is raised during the system run,
        BankAccount.del_all_accounts()          #it will delete memory of all accounts upon system exit,
        print('The program will now close.\n')  #print this message, and
        return                                  #return nothing, making it so that the kernel doesn't die.

#########################################################

#####  DOUBLE CHECK FOR BUGS IN ANYTHING
#####  MAKE A DETAILED REPORT WITH SCREENSHOTS OF OUTPUT

system_start()
print(account_101)

     *
    ***
   *****
  *******
 *********
***********
 *********
  *******
   *****
    ***
     *

Bank Management System

Username: BIOS6642
Password: python
Login Successful.

Main Menu
Type "1" to display all accounts
Type "2" for account statistics
Type "3" for displaying accounts in subset of a category
Type "4" for account query
Type "5" to open a new account
Type "6" to delete an account
Type "7" to conduct a transaction (deposit, withdrawal, transfer, display balance,
change name/age/residency, display recent account activity)
Type "q" to quit.
Selection: 4
Please enter account number or name of account owner(type "m" to return to main menu, or "q" to quit): 101

< Account number  Name       Age  Residency    Balance      Date       >
< 101             Bert       32   California   766209.0     02/23/11   >

Main Menu
Type "1" to display all accounts
Type "2" for account statistics
Type "3" for displaying accounts in subset of a category
Type "4" for account query
Type "5" t

NameError: name 'account_101' is not defined